# Exploring Evaluation Datasets

In [10]:
# Imports
import sqlite3
import pandas as pd
import json
import re
import random
from collections import Counter
from IPython.display import clear_output

import sys
sys.path.insert(0, "../")

from app.backend.processing.process_query.query_pipeline import transform_query
from data.annotation.benchmark_creation import categorize_spider
from utils.query_pattern_recognition import ExtractException, QueryInfo

## WikiSQL

In this notebook we focus at:
* Exploring WikiSQL
* Loading it into a sqlite3 db
* Checking if the query results can be parsed it into single table, single row ToTTo expected format

In [63]:
# Constants
WIKISQL_DB_PATH = "../storage/datasets/wiki_sql/raw/train.db"
WIKISQL_JSON_PATH = "../storage/datasets/wiki_sql/raw/train.tables.jsonl"
QUERIES_PATH = "../storage/datasets/wiki_sql/raw/train.jsonl"

## Reading WikiSQL

In [10]:
def run_query_on_connection():
    pass

def connect_and_query(db_path, query):
    con = sqlite3.connect(db_path)
    cur = con.cursor()

    cur.execute(query)
    res = cur.fetchall()
    con.close()
    
    return res

In [12]:
# Check the table names in the database
print(connect_and_query(WIKISQL_DB_PATH, "SELECT name FROM sqlite_master WHERE type='table';")[:5])

[('table_1_1000181_1',), ('table_1_10006830_1',), ('table_1_10007452_3',), ('table_1_10015132_1',), ('table_1_10015132_14',)]


As we can see the table names have been given a special id. Unfortunately, this leads to lost information that could be used in explaining the query.

In [22]:
def concat_col_names_and_res(res, col_names):
    concatenated = [col_names]
    for row in res:
        concatenated.append(row)
    
    return concatenated

pd.DataFrame(concat_col_names_and_res(*query_with_col_names(WIKISQL_DB_PATH, "SELECT * FROM table_1_10007452_3;")))

,0,1,2,3,4,5
0,col0,col1,col2,col3,col4,col5
1,1992-93,gillig,phantom (high floor),444-464 (21),dd s50egr allison wb-400r,diesel
2,1996,gillig,phantom (high floor),465-467 (3),dd s50 allison wb-400r,diesel
3,1998,gillig,phantom (high floor),468-473 (6),dd s50 allison wb-400r,diesel
4,2000,gillig,advantage (low floor),474-481 (8),cummins isc allison wb-400r,diesel
5,2002,gillig,advantage (low floor),482-492 (11),cummins isl allison wb-400r,diesel
6,2010,nfi,ge40lfr,300-309 (10),ford triton v10 ise-thundervolt tb40-hg,hybrid
7,2011,nfi,c40lfr,310-329 (20),cummins westport isl-g allison wb-400r,cng


In [57]:
class wikiqsl_controller:
    def __init__(self, db_path, tables_path):
        self.db_path = db_path
        
        self.table_info = []
        with open(tables_path) as file_in:
            for line in file_in:
                self.table_info.append(json.loads(line))

    def connect_and_query(self, query):
        con = sqlite3.connect(self.db_path)
        cur = con.cursor()

        cur.execute(query)
        res = cur.fetchall()
        con.close()

        return res
    
    def find_json_table(self, table_id):
        for table in self.table_info:
            if table['id'] == table_id:
                return table
            
    def find_col_names(self, table_id):
        return self.find_json_table(table_id)['header']
    
    def query_with_title(self, query):
        try:
            extracted_table_id = re.search('FROM\stable_(\w*)[\s|;]', query).group(1)
            extracted_table_id = extracted_table_id.replace("_", "-")
        except AttributeError as e:
            raise AttributeError(f"TableId could not be extracted from {query}")
        
        res = self.connect_and_query(query)
        col_names = self.find_col_names(extracted_table_id)
        res.insert(0, tuple(col_names))
        
        return res


In [59]:
wikisql = wikiqsl_controller(WIKISQL_DB_PATH, WIKISQL_JSON_PATH)
pd.DataFrame(wikisql.query_with_title("SELECT * FROM table_1_10007452_3;"))

,0,1,2,3,4,5
0,Order Year,Manufacturer,Model,Fleet Series (Quantity),Powertrain (Engine/Transmission),Fuel Propulsion
1,1992-93,gillig,phantom (high floor),444-464 (21),dd s50egr allison wb-400r,diesel
2,1996,gillig,phantom (high floor),465-467 (3),dd s50 allison wb-400r,diesel
3,1998,gillig,phantom (high floor),468-473 (6),dd s50 allison wb-400r,diesel
4,2000,gillig,advantage (low floor),474-481 (8),cummins isc allison wb-400r,diesel
5,2002,gillig,advantage (low floor),482-492 (11),cummins isl allison wb-400r,diesel
6,2010,nfi,ge40lfr,300-309 (10),ford triton v10 ise-thundervolt tb40-hg,hybrid
7,2011,nfi,c40lfr,310-329 (20),cummins westport isl-g allison wb-400r,cng


## Parsing Queries

In [94]:
def sql_query_creator(table_id, sel_index, agg_index, conditions):
    agg_ops = ['', 'MAX', 'MIN', 'COUNT', 'SUM', 'AVG']
    cond_ops = ['=', '>', '<', 'OP']
    
    table_id = f"table_{table_id.replace('-', '_')}"
    
    rep = f"SELECT {agg_ops[agg_index]}(col{sel_index}) " if agg_index != 0 else f"SELECT col{sel_index} "
    rep += f"FROM {table_id} "

    if conditions:
        rep +=  'WHERE ' + ' AND '.join(['{} {} "{}"'.format('col{}'.format(i), cond_ops[o], v) for i, o, v in conditions])
    return rep

In [64]:
queries = []
with open(QUERIES_PATH) as file_in:
    for line in file_in:
        queries.append(json.loads(line))

In [99]:
json_query = queries[4]
query = sql_query_creator(json_query['table_id'], json_query['sql']['sel'], json_query['sql']['agg'], json_query['sql']['conds'])
print(query, wikisql.query_with_title(query))

SELECT COUNT(col3) FROM table_1_10007452_3 WHERE col5 = "CNG" [('Order Year', 'Manufacturer', 'Model', 'Fleet Series (Quantity)', 'Powertrain (Engine/Transmission)', 'Fuel Propulsion'), (0,)]


## Annotator

In [ ]:
class Annotator:
    def __init__(self, db_path, tables_path, queries_path):
        self.db_path = db_path

## Spider

In [2]:
QUERIES_PATH = "../storage/datasets/spider/original/train_spider.json"
DB_DIR = "../storage/datasets/spider/original/database/"
ANNOTATED_QUERIES_DIR = "../storage/datasets/spider/annotations/queries/"

In [6]:
with open(QUERIES_PATH, 'r') as file:
    queries = json.load(file)

original_queries_numb = len(queries)
# Filter out nested queries
# queries = [q for q in queries if len(q['query'].split('SELECT')) == 2 and 'JOIN' not in q['query']]
queries = [q for q in queries if len(q['query'].split('SELECT')) == 2 and ('time' not in q['query'].lower())]

print(f"Remaining queries: {len(queries)}/{original_queries_numb}\n")
    
for q in random.sample(queries, 5):
    print(f"DB: {q['db_id']}")
    print(f"Query: {q['query']}")
    print(f"Question: {q['question']}")
    print()

Remaining queries: 5933/7000

DB: voter_2
Query: SELECT avg(Age) FROM STUDENT WHERE Sex  =  "F"
Question: What is the average age of female (sex is F) students?

DB: device
Query: SELECT T2.Carrier FROM stock AS T1 JOIN device AS T2 ON T1.Device_ID  =  T2.Device_ID GROUP BY T1.Device_ID HAVING COUNT(*)  >  1
Question: What are the carriers of devices that are in stock in more than a single shop?

DB: cre_Theme_park
Query: SELECT other_hotel_details ,  star_rating_code FROM HOTELS ORDER BY price_range ASC LIMIT 3
Question: Show the details and star ratings of the 3 least expensive hotels.

DB: program_share
Query: SELECT t1.name FROM program AS t1 JOIN broadcast AS t2 ON t1.program_id  =  t2.program_id GROUP BY t2.program_id ORDER BY count(*) DESC LIMIT 1
Question: Which program is broadcast most frequently? Give me the program name.

DB: icfp_1
Query: SELECT fname FROM authors WHERE lname  =  "Ueno"
Question: What is the first name of the author with last name "Ueno"?



In [4]:
def connect_and_query(db_path, query):
    con = sqlite3.connect(db_path)
    
    cur = con.cursor()
    cur.execute(query)
    res = cur.fetchall()
    desc = [d[0] for d in cur.description]
    
    con.close()
    
    return res, desc
    
    
def create_db_path(db_dir, db_id):
    return f"{db_dir}{db_id}/{db_id}.sqlite"


def print_query_info(query, transformed_query, res, cols):
    print()
    print("#" * 120)
    print(f"DB: {query['db_id']}")
    print(f"Original Query: {query['query']}")
    print(f"Transformed Query: {transformed_query}")
    print(f"Question: {query['question']}")
    print()
    display(pd.DataFrame(res, columns=cols))

In [7]:
for ind, q in enumerate(queries):
#     if ind > 5250:
    clear_output(wait=True)
    print(ind, q['query'])
    _, _ = transform_query(q['query'])
    

5932 SELECT T2.company_name FROM movie AS T1 JOIN culture_company AS T2 ON T1.movie_id  =  T2.movie_id WHERE T1.year  =  1999


In [5]:
last_ind = -1

for q in random.sample(queries, 500):
    clear_output(wait=True)
    try:
        transformed_query, _ = transform_query(q['query'])
    except KeyError:
        raise KeyError(f"Failed to transform {q['query']}")
    transformed_query = transformed_query[:-1] + '3'
    
    res, cols = connect_and_query(
        create_db_path(DB_DIR, q['db_id']),
        transformed_query
    )
    print_query_info(q, transformed_query, res, cols)
    
    
    store_ind = int(input(f"Store results file ind (previous: {last_ind}): "))
    if store_ind == -1:
        continue
    else:
        pd.DataFrame(res, columns=cols).to_csv(f"{ANNOTATED_QUERIES_DIR}q{store_ind}.csv", index=False)
        last_ind = store_ind
    


########################################################################################################################
DB: club_1
Original Query: SELECT t3.lname FROM club AS t1 JOIN member_of_club AS t2 ON t1.clubid  =  t2.clubid JOIN student AS t3 ON t2.stuid  =  t3.stuid WHERE t1.clubname  =  "Bootup Baltimore"
Transformed Query: SELECT t3.lname AS "student lname", t1.clubname AS "club clubname" FROM club AS t1 JOIN member_of_club AS t2 ON t1.clubid = t2.clubid JOIN student AS t3 ON t2.stuid = t3.stuid WHERE t1.clubname = 'Bootup Baltimore' LIMIT 3
Question: Find the last names of the members of the club "Bootup Baltimore".



,student lname,club clubname
0,Smith,Bootup Baltimore
1,Lee,Bootup Baltimore


KeyboardInterrupt: Interrupted by user

In [31]:
db_name = "network_2"
query = "SELECT * FROM Person LIMIT 2"

res, cols = connect_and_query(create_db_path(DB_DIR, db_name), query)

pd.DataFrame(res, columns=cols)

,name,age,city,gender,job
0,Alice,25,new york city,female,student
1,Bob,35,salt lake city,male,engineer


In [14]:
queries_sql = [{'query': q['query']} for q in queries]

with open('../tests/resources/spider_queries.json', 'w') as outfile:
    json.dump(queries_sql, outfile)

### Benchmark Analysis


In [7]:
SPIDER_TRAIN_PATH = "../storage/datasets/spider/original/train_spider.json"

with open(SPIDER_TRAIN_PATH, 'r') as file:
    train_datapoints = json.load(file)

categorized_datapoints = categorize_spider(train_datapoints)

#### Queries per category

In [12]:
categories = [datapoint['category'] for datapoint in categorized_datapoints]
counted_categories = Counter(categories)
counted_categories

Counter({'aggregate': 720,
         'large_select': 98,
         'small_select': 872,
         'aggregate_group_by': 874,
         'join': 1055,
         'join_aggregate': 876})

#### Number of JOINs

In [19]:
join_datapoints = [datapoint for datapoint in categorized_datapoints 
                   if datapoint['category'] == 'join']

joins_numb = []

for datapoint in join_datapoints:
    query_info = QueryInfo(datapoint['query'])
    joins_numb.append(query_info.joins['num'])
    
counted_joins = Counter(joins_numb)
counted_joins

Counter({1: 718, 2: 303, 3: 30, 4: 4})

#### SELECT size of JOINs

In [20]:
join_datapoints = [datapoint for datapoint in categorized_datapoints 
                   if datapoint['category'] == 'join']

select_sizes = [QueryInfo(datapoint['query']).select['columns_num'] for datapoint in join_datapoints]

counted_selects = Counter(select_sizes)
counted_selects

Counter({2: 356, 1: 616, 3: 64, 4: 13, 0: 4, 5: 2})